In [1]:
import PIL

# エラーなくPILが通るはず

In [2]:
# uninstallします
!pip uninstall pillow -y

Found existing installation: Pillow 9.4.0
Uninstalling Pillow-9.4.0:
  Successfully uninstalled Pillow-9.4.0


一度、ランタイムを再起動してください（PILが残っている）

---

In [1]:
import PIL

# エラーが出るはずです　No module named 'PIL'

ModuleNotFoundError: ignored

In [2]:
# [1] 生成されたコードが例えば以下だったとします。
# 単純に画像を取得して、幅と高さを求めているだけです。

generated_code_txt="""
from PIL import Image
import requests
from io import BytesIO

# 適当な画像のURLを指定
image_url = 'https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F3328834%2Ff8a26020-6ec7-aa41-9ff6-0c4d38cb8319.png?ixlib=rb-4.0.0&auto=format&gif-q=60&q=75&w=1400&fit=max&s=521ce0ce7dc4fe5865bc41b79a4b735f'

# 画像を取得
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# 画像を取得
response = requests.get(image_url)
img = Image.open(BytesIO(response.content))

# 画像の幅と高さを取得
width, height = img.size

# 幅と高さを表示
print("幅:", width)
print("高さ:", height)
"""

生成コードをそのまま実行せず、try-exceptで挟むようにします。

またExecptionの例外発生時には、そのエラーメッセージ（スタックトレース）を取得するようにします

In [3]:
def add_try_except_block(code_str):
    """try-exceptでコードを挟む関数です"""
    indented_code = "\n".join(["    " + line for line in code_str.splitlines()])

    try_except_code = f"""
try:
    traceback_str = None
{indented_code}
except Exception as e:
    import traceback
    traceback_str = traceback.format_exc()"""
    return try_except_code

In [4]:
# [1] 生成されたコードをtry-exceptで挟む
generated_code_txt_with_try = add_try_except_block(generated_code_txt)

# [2] try-exceptで挟んだ状態を確認
print(generated_code_txt_with_try)



try:
    traceback_str = None
    
    from PIL import Image
    import requests
    from io import BytesIO
    
    # 適当な画像のURLを指定
    image_url = 'https://qiita-user-contents.imgix.net/https%3A%2F%2Fqiita-image-store.s3.ap-northeast-1.amazonaws.com%2F0%2F3328834%2Ff8a26020-6ec7-aa41-9ff6-0c4d38cb8319.png?ixlib=rb-4.0.0&auto=format&gif-q=60&q=75&w=1400&fit=max&s=521ce0ce7dc4fe5865bc41b79a4b735f'
    
    # 画像を取得
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    
    # 画像を取得
    response = requests.get(image_url)
    img = Image.open(BytesIO(response.content))
    
    # 画像の幅と高さを取得
    width, height = img.size
    
    # 幅と高さを表示
    print("幅:", width)
    print("高さ:", height)
except Exception as e:
    import traceback
    traceback_str = traceback.format_exc()


---

文字列で書かれたPythonコードを実行する機会は、（私は）コード生成以外の場面で遭遇したことがなく、コードを読めばある程度挙動は予測できますが、実行するのは非常に怖いです。


exec()で文章全体を実行したり、

eval()で1文ずつ実行することができますが、

より安全なのは、ast（Abstract Syntax Tree、抽象構文木）パッケージを使用し、

ast.literal_eval()で実行する方法になります。

https://zenn.dev/sergicalsix/articles/92b9526e20c7a6
https://qiita.com/tkmz/items/717d524083b71a4af75f

とはいえ、LLMでのコード生成の時代になり、生成されたコードに
本当に害がないかの判断はこれだけでは心配です。


「バリデーション」しようにも、生成コードの何が妥当で何は妥当ではないかは難しいです。

「エスケープ処理（≒サニタイズ）」しようにも、生成コードとして実は意味がある可能性があるので、文字列変換して良いのか判断つかない場合があります。

ファイルの削除につながる系はブラックリスト型でチェックを入れるくらいは想像できるのですが、それ以上になると、なかなか。。。











---

tracebackパッケージのtraceback.print_exc()、もしくは traceback.format_exc()を実行することで、エラーの文章（traceback）を表示させ、その文章をリトライ時の入力にしようすることにします





In [5]:
# パッケージのインストール
!pip install openai==0.28

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.6 MB/s eta 0:00:00


非常に危険。API_KEYを直書き

In [6]:
import os
import re
import openai


KEY = "sk-hogehoge"
openai.api_key = KEY

In [7]:
def extract_python_code(text):
    """pythonの実装のみを抽出"""
    pattern = r'```python(.*?)```'
    result = re.findall(pattern, text, re.DOTALL)
    # result リスト内のすべての要素を結合して1つの文字列にする
    extracted_code = '\n'.join(result)
    return extracted_code

In [8]:
def get_response_from_chatGPT(generated_code_txt_with_try, traceback_str):

    # Promptの作成
    user_content = f"""
以下のPythonコードを実行したところ、次のようなスタックトレースが表示されました。
再度このPythonコードを実行し、エラーなく完了させるためには、このスタックトレースの内容を解決するために、
最低限として事前にどのようなPythonコードを実行する必要がありますか？
Pythonのコード形式で教えてください。pipコマンドなどが必要な場合は、subprocess.runを使用し、標準出力をprintするように、

output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT, universal_newlines=True)
print(output)
を使用してください。

### [Pythonコード]
{generated_code_txt_with_try}

###  [スタックトレース]
{traceback_str}

### [事前に実行するPythonコード]
"""

    messages = [
      {"role": "system","content":"You are an AI assistant that helps people find information."},
      {"role": "user", "content":user_content}
    ]


    # responseを得る
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages = messages,
        temperature=0.0,
        top_p=0.0,
        max_tokens=800,
        )

    # Pythonコードのみを取り出す
    ret = response['choices'][0]['message']['content']  # 回答
    correction_code = extract_python_code(ret)

    return correction_code

In [9]:
def main(generated_code_txt):

  # [1] 何らかコードを生成したとします
  # generated_code_txt

  # [2] try-exceptで挟み、スタックトレースをとれるようにします
  generated_code_txt_with_try = add_try_except_block(generated_code_txt)

  # [3] エラーがなくなるまで回します（本当は制限回数を定めるべき）
  error_is = True

  while error_is:
      # [3-1] 空の辞書を作成。exec実行した中身の変数をここに格納します
      variables = {}
      traceback_str= None

      # [3-2] 実行 execを使用してコードを実行し、exec内の変数を辞書で取得します
      print("============================================")
      print("[1] 生成されたコードを実行します（今回は適当な画像のサイズを出力するコードです）\n")
      exec(generated_code_txt_with_try, globals(), variables)

      # [3-3] エラーなく終わったかを判定します
      if  not variables.get("traceback_str"):
          error_is = False  # loopから抜ける
          print("\n[Fin] 実行が完了しました")
          break

      # [3-4] エラーが発生し、スタックトレースが存在する場合、それを表示します
      if  variables.get("traceback_str"):
          print("[2] エラーが発生しました\n\n")
          print(traceback_str, variables.get("traceback_str"))

      # [3-5] 修正コマンドの実行
      print("\n[3] 修正方法を考えます。少々お待ちください。\n")
      correction_code = get_response_from_chatGPT(generated_code_txt_with_try, traceback_str)

      # [3-6] 実行して良いかの確認
      print("\n[4] 修正のために、以下のコマンドを実行しても良いですか？\n")
      print(correction_code)
      user_input = input("宜しければyを入力しEnterを押してください。実行を中止する場合はnを入力しEnterを押してください: ")

      # [3-7] 修正の実行
      if user_input.lower() == 'y':
          print("\n[5] それでは修正します。\n")
          exec(correction_code, globals())
          print("[6] 修正が完了しました\n\n 再実行に入ります・・・\n\n")

      else:
          print("ユーザーが実行中止を選択しました。")
          break


In [10]:
# メインの実行
main(generated_code_txt_with_try)

[1] 生成されたコードを実行します（今回は適当な画像のサイズを出力するコードです）

[2] エラーが発生しました


None Traceback (most recent call last):
  File "<string>", line 8, in <module>
ModuleNotFoundError: No module named 'PIL'


[3] 修正方法を考えます。少々お待ちください。


[4] 修正のために、以下のコマンドを実行しても良いですか？


import subprocess

command = 'pip install pillow'
output = subprocess.check_output(command, shell=True, stderr=subprocess.STDOUT, universal_newlines=True)
print(output)

宜しければyを入力しEnterを押してください。実行を中止する場合はnを入力しEnterを押してください: y

[5] それでは修正します。

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 13.7 MB/s eta 0:00:00

[6] 修正が完了しました

 再実行に入ります・・・


[1] 生成されたコードを実行します（今回は適当な画像のサイズを出力するコードです）

幅: 1322
高さ: 715

[Fin] 実行が完了しました


In [ ]:
# 以上